In [ ]:
# Testing ml_processor

In [1]:
from tensorflow import keras

In [2]:
# This function will load a model based on the path and model_name provided
def load_model(model_name: str, path="./ml_model"):
    try:
        model_path = f"{path}/{model_name}.keras"
        loaded_model = keras.models.load_model(model_path)
        return loaded_model
    except Exception as e:
        print(e)

In [3]:
loaded_model = load_model('dmd_lstm')

In [4]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [10]:
import numpy as np

In [20]:
def process_data(loaded_model, len_predictions=5):
    try:
        # Iterate through the 20 stock data files from /data/db/stock_data/{stock_symbol}.csv
        stock_symbols = stock_symbols = [
    'MEG',
    'JGS',
    'BDO',
    'FGEN',
    'ICT',
    'ALI',
    'SMC',
    'TEL',
    'GLO',
    'BLOOM',
    'RLC',
    'MER',
    'AC',
    'PGOLD',
    'LTG',
    'MPI',
    'AP',
    'RRHI',
    'URC'
]
        # Appeend PSEI to the list of stock symbols
        stock_symbols.append("PSEI")

        # Initialize the dictionary that will hold the processed data
        # key = stock symbol
        # value = (last actual closing, list of predictions with length=predict_len)
        processed_data = {stock:[] for stock in stock_symbols}

        # Iterate through the stock symbols
        for stock in stock_symbols:
            predicted_closings = []
            
            # Initialize the closing prices of the stock
            closing_prices = np.array([])
            # Get the last closing prices based on the window size of the model
            window_size = loaded_model.count_params() - 1
            closing_prices = np.genfromtxt(f'./ml_model/data_test/{stock}.csv',
                                            delimiter=',',
                                            skip_header=1,
                                            usecols=4)
            closing_prices = closing_prices[-(window_size):]
            # Get the last actual closing price
            last_actual = closing_prices[-1]

            """
                PREDICTED CLOSINGS:
                    1st Prediction: input[closing_prices]
                    2nd Prediction: input[closing_prices[-4:], pred]
                    3rd Prediction: input[closing_prices[-3:], pred]
                    4th Prediction: input[closing_prices[-2:], pred]
                    5th Prediction: input[closing_prices[-1:], pred]
            """
            for _ in range(len_predictions):
                data = np.append(closing_prices, predicted_closings)
                prediction = loaded_model.predict(data[-(window_size):].reshape(1,5))
                predicted_closings.append(float(prediction))

            processed_data[stock] = tuple((last_actual, predicted_closings))

        return processed_data
    
    except Exception as e:
        print(e)

In [21]:
processed_data = process_data(loaded_model=loaded_model)

1/1 [==============================] - 0s 23ms/step


In [22]:
processed_data

{'MEG': (2.34,
  [2.3027100563049316,
   2.372716188430786,
   2.3127152919769287,
   2.342719316482544,
   2.3054213523864746]),
 'JGS': (53.9,
  [53.902706146240234,
   54.002685546875,
   53.602657318115234,
   53.90264892578125,
   53.905338287353516]),
 'BDO': (125.5,
  [124.00247955322266,
   126.00253295898438,
   125.00251007080078,
   125.50272369384766,
   124.00498962402344]),
 'FGEN': (18.02,
  [18.382694244384766,
   18.10270118713379,
   18.182720184326172,
   18.022674560546875,
   18.385387420654297]),
 'ICT': (214.0,
  [206.6023406982422,
   212.00233459472656,
   211.40199279785156,
   214.00294494628906,
   206.60494995117188]),
 'ALI': (29.25,
  [29.052684783935547,
   29.202686309814453,
   29.102670669555664,
   29.2526912689209,
   29.05536651611328]),
 'SMC': (107.5,
  [114.60272216796875,
   112.70266723632812,
   110.00323486328125,
   107.5022964477539,
   114.60485076904297]),
 'TEL': (1380.0,
  [1410.0040283203125,
   1395.0025634765625,
   1380.00341796875